# Part 2: Illness Detection
### This part determines level of illness within sheep based on sitting/standing ground truth data

In [13]:
# Imports
import pandas as pd
import numpy as np

In [16]:
# read in the data first
# note that D is missing several days
files = ['A.csv', 'B.csv', 'U.csv', 'V.csv', 'W.csv', 'D.csv']
sheep = []

for file in files:
    df = pd.read_csv(file)
    # drop unnecessary columns
    df = df.drop(columns=df.columns[3:])
    df = df.drop(columns=df.columns[0:2])

    # rename column
    df.columns = ['Standing']

    # change accelerometer value to 1 if standing, 0 if sitting
    df['Standing'] = np.where(df['Standing'] < -0.4, 1, 0)
    sheep.append(df)

Standing    1
Name: 141, dtype: int64
